# Enrichment Analysis of Gene Attribution Scores

In this notebook, we analyze the attribution scores obtained from the CLIP gene expression embedding space with the script `interpretability.py`.
We screen the genes with highest attribution scores for enrichment of Gene Ontology Cellular Component Terms with Gene Set Enrichment Analysis Preranked.

In [1]:
import numpy as np
import pandas as pd
import gseapy as gp

Load the attribution score dataframe

In [2]:
gene_scores = pd.read_csv("../data/attribution_scores/genes45.csv", index_col=0)

In [3]:
def get_common_set(df, n_genes=100, method="union"):
    '''
    Get the union or intersection of top `n_genes` genes over all reference clusters for each query cluster
    '''
    gene_sets = {}
    for i in range(5):
        gene_sets[i] = set()
        first = True
        for j in range(5):
            if j != i:
                glist = df[f"{i} vs. {j}"].sort_values(ascending=False).iloc[:n_genes].index.tolist()
                if method == "union":
                    gene_sets[i] = gene_sets[i].union(glist)
                elif method == "intersection":
                    if first:
                        gene_sets[i] = set(glist)
                        first = False
                    else:
                        gene_sets[i] = gene_sets[i].intersection(glist)
    
    return gene_sets

def gsea_prerank(ig_df: pd.DataFrame, n_genes: int = 100):
    '''
    Run GSEA Prerank on the top `n_genes` highest attributed genes for each query-reference combination.
    '''
    cluster_names = set([int(c.split()[0]) for c in ig_df.columns])
    results = {}
    for i in range(5):
        for j in range(5):
            if i != j:
                # Get n_genes highest scored genes
                top = ig_df[f"{j} vs. {i}"].sort_values(ascending=False).iloc[:n_genes]
                top.index = top.index.str.upper()

                # Run GSEA Prerank
                gene_set = 'GO_Cellular_Component_2023'
                results[f"{j} vs. {i}"] = gp.prerank(rnk = top, gene_sets = gene_set, seed=1, outdir=None, min_size=1).res2d
    return results


Run GSEA Preranked and concatenate the enriched terms of all query-reference pairs

In [4]:
N_GENES = 100

gsea = gsea_prerank(gene_scores, N_GENES)

for k,g in gsea.items():
    g["query_reference"] = k
    
terms = pd.concat(list(gsea.values()), axis=0, ignore_index=True)
terms["n_lead_genes"] = terms["Lead_genes"].apply(lambda x: len(x.split(";")))

Filter enriched terms based on significance level and number of lead genes

In [5]:
ALPHA = 0.02
MIN_LEAD_GENES = 4

terms = terms[terms["NOM p-val"] < ALPHA]
terms = terms[terms["n_lead_genes"] >= MIN_LEAD_GENES]

In [6]:
terms

,Name,Term,ES,NES,NOM p-val,FDR q-val,FWER p-val,Tag %,Gene %,Lead_genes,query_reference,n_lead_genes
0,prerank,Lysosome (GO:0005764),-0.602151,-1.926029,0.007968,0.059244,0.027,7/7,45.00%,ENPEP;DNAJC13;CUBN;LRP2;SLC30A2;DAB2;ARSB,1 vs. 0,7
642,prerank,Basolateral Plasma Membrane (GO:0016323),0.852632,2.047549,0.0,0.008479,0.006,5/5,19.00%,AQP4;ATP1B3;EPCAM;SLC4A11;ANXA2,4 vs. 1,5
1081,prerank,Endoplasmic Reticulum Lumen (GO:0005788),0.717999,2.022298,0.002587,0.003993,0.003,6/9,16.00%,TNC;SPON1;IGFBP5;PDGFA;THBS1;LTBP1,4 vs. 2,6
1082,prerank,Intracellular Organelle Lumen (GO:0070013),0.673927,2.008438,0.002475,0.003194,0.006,7/11,18.00%,TNC;SPON1;IGFBP5;PDGFA;THBS1;LTBP1;MUC1,4 vs. 2,7
1762,prerank,Basolateral Plasma Membrane (GO:0016323),0.655914,1.834001,0.010014,0.108331,0.078,7/7,39.00%,SLC39A8;SLC22A7;SLC6A13;AQP7;UMOD;LRRK2;SLC22A2,2 vs. 4,7
1859,prerank,Intracellular Non-Membrane-Bounded Organelle (...,0.734317,1.936694,0.0,0.000428,0.001,8/10,25.00%,CIDEA;DGAT2;PLIN1;ADIG;CAV1;FABP4;CIDEC;LIPE,3 vs. 4,8
1860,prerank,Lipid Droplet (GO:0005811),0.734317,1.936694,0.0,0.000428,0.001,8/10,25.00%,CIDEA;DGAT2;PLIN1;ADIG;CAV1;FABP4;CIDEC;LIPE,3 vs. 4,8
